# MNIST 손글씨 - CNN v4
- 2 Conv2D layers ==> 2 Conv2D with padding layers
    - ex. (4,4)크기의 입력에 (3,3) 크기의 커널을 적용한다면, (2,2) 크기의 특성맵을 만듦
    -       커널 크기는 (3,3) 그대로 두고 출력의 크기를 입력과 동일하게 (4,4)로 만들기 위해 사용
    -       입력 배열의 주위를 가상의 원소로 채우는 것을 패딩이라고 한다. (실제값은 0으로 채워져 있기 때문에 계산에 영향을 미치지 않는다.)
    - 모서리에 있는 중요한 정보가 특성 맵으로 잘 전달되지 않을 가능성이 높고, 가운데 있는 정보는 두드러지게 표현한다. 
- 1 MaxPooling2D layer
- 1 Dropout(0.25) layer 
- 1 Fully Connected Network layer
- 1 Dropout(0.5)
- 1 Output layer

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

- 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# X data: 0~1 사이의 값, 3차원 --> 4차원
X_train = X_train.reshape(-1,28,28,1) / 255.
X_test = X_test.reshape(-1,28,28,1) / 255.

In [4]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

- 모델 정의/학습/평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),   
    MaxPooling2D(),                           
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 512)               6423040   
                                                        

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model_path = 'best-mnist-cnn-v4.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [9]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=200,
    callbacks=[checkpoint, early_stop], verbose=0
)


Epoch 00001: val_loss improved from inf to 0.05757, saving model to best-mnist-cnn-v4.h5

Epoch 00002: val_loss improved from 0.05757 to 0.04779, saving model to best-mnist-cnn-v4.h5

Epoch 00003: val_loss improved from 0.04779 to 0.04147, saving model to best-mnist-cnn-v4.h5

Epoch 00004: val_loss improved from 0.04147 to 0.03507, saving model to best-mnist-cnn-v4.h5

Epoch 00005: val_loss did not improve from 0.03507

Epoch 00006: val_loss improved from 0.03507 to 0.03353, saving model to best-mnist-cnn-v4.h5

Epoch 00007: val_loss did not improve from 0.03353

Epoch 00008: val_loss did not improve from 0.03353

Epoch 00009: val_loss improved from 0.03353 to 0.03229, saving model to best-mnist-cnn-v4.h5

Epoch 00010: val_loss did not improve from 0.03229

Epoch 00011: val_loss did not improve from 0.03229

Epoch 00012: val_loss did not improve from 0.03229

Epoch 00013: val_loss did not improve from 0.03229

Epoch 00014: val_loss did not improve from 0.03229

Epoch 00015: val_loss d

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0258 - accuracy: 0.9919


[0.025845658034086227, 0.9919000267982483]